In [1]:
import MagmaPandas.geochemistry.volatile_solubility.IaconoMarziano as im
import MagmaPandas.geochemistry.volatile_solubility.Allison2022 as a
im_config = im.IaconoMarziano_configuration()
im_config.parameters = "hydrous_webapp"
im_config.print()

# Iacono-Marziano volatile solubility ##
Settings________________________________
Parameterisation..........hydrous_webapp
Fugacity model.....................ideal
Activity model.....................ideal
Species model......................mixed

Calibration range______________________
Temperature..................1373-1673°K
Pressure.....................0.1-10 kbar


In [2]:
# import pandas as pd
import MagmaPandas as mp
import MagmaPandas.parse.readers as r
import pandas as pd
import elements as e
import numpy as np
import matplotlib.pyplot as plt
import VESIcal as v
config = mp.configuration()

/Users/thomas/miniforge3/lib/python3.9/site-packages/VESIcal/calculate_classes.py:7: UserWarning: 

  from VESIcal.models import magmasat


In [3]:
melt = mp.read_melt_inclusion('./melt.csv', index_col=['name'], total_col='total')

In [4]:
sample = "PI052-02-01"
FeO_initial = 15
Fo_host = 0.6
Kd = 0.29
P_bar = 1e3
T_K = 1400
inclusion = melt.iloc[4,:].copy()
# inclusion["H2O"] = 5


In [5]:
s = v.Sample(inclusion[inclusion.index.intersection(v.oxides)])
s.get_composition()

SiO2     47.851799
Al2O3    15.871700
MgO       7.863920
CaO      10.468700
FeO       8.851050
Na2O      3.148020
K2O       1.180670
MnO       0.098343
TiO2      3.431850
P2O5      0.598803
Cr2O3          NaN
CO2       0.357400
H2O       1.046258
dtype: float32

In [7]:
for idx in range(20):

    inclusion = melt.iloc[idx,:].copy()
    s = v.Sample(inclusion[inclusion.index.intersection(v.oxides)])
    model = "IaconoMarziano"
    VC = v.calculate_saturation_pressure(s, temperature=T_K, model=model).result
    IM, xim = im.calculate_saturation(inclusion, T_K, output="both")
    A, x = a.calculate_saturation(inclusion, T_K, output="both")
    string = f"IM: {IM:.2f}, A: {A:.2f}"
    print(f"VESIcal: {VC:<30.2f} {string:>25}, delta: {(VC - IM): .0f} bar, Xfl: {x:.3f}, Xfl-im: {xim: .3f}")

VESIcal: 1135.08                          IM: 1107.76, A: 1460.26, delta:  27 bar, Xfl: 0.032, Xfl-im:  0.042
VESIcal: 980.97                            IM: 959.98, A: 1582.02, delta:  21 bar, Xfl: 0.030, Xfl-im:  0.050
VESIcal: 2697.32                          IM: 2569.67, A: 4117.28, delta:  128 bar, Xfl: 0.028, Xfl-im:  0.057
VESIcal: 3550.70                          IM: 3392.23, A: 2098.91, delta:  158 bar, Xfl: 0.028, Xfl-im:  0.022
VESIcal: 4161.33                          IM: 3936.30, A: 3922.23, delta:  225 bar, Xfl: 0.027, Xfl-im:  0.040
VESIcal: 1705.94                          IM: 1651.75, A: 1945.10, delta:  54 bar, Xfl: 0.057, Xfl-im:  0.083
VESIcal: 2065.49                          IM: 1989.94, A: 3028.65, delta:  76 bar, Xfl: 0.005, Xfl-im:  0.011
VESIcal: 2155.12                          IM: 2075.37, A: 1639.88, delta:  80 bar, Xfl: 0.072, Xfl-im:  0.058
VESIcal: 3190.00                          IM: 3036.59, A: 4077.36, delta:  153 bar, Xfl: 0.029, Xfl-im:  0.057
VESIca

In [ ]:
model = "IaconoMarzianoCarbon"

p = np.arange(1,  1e4, 5e1)
fig, ax = plt.subplots(figsize=(10, 10))

plt.plot(p, [im.co2.calculate_solubility(inclusion, P_bar=i, T_K=T_K) for i in p])
plt.plot(p, [v.calculate_dissolved_volatiles(s, temperature=T_K-273.15, pressure=i, model=model).result for i in p], label="VC")

plt.legend()
plt.show()